GMI-атака

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import os, shutil
import torch.nn.functional as F
import torchvision.utils as vutils
import numpy as np

dataloader

In [2]:
# public data
public_data_path = "./background/MNIST/trainingSet/trainingSet/public"
# private data
private_data_path = "./background/MNIST/trainingSet/trainingSet/private" 

In [3]:
from torch.nn.modules.loss import _Loss
import torch.utils.data as data
from torchvision import transforms
import PIL
import torch.nn.functional as F 
import os

class GrayFolder(data.Dataset):
    def __init__(self, img_path, mode, n_classes):
        self.mode = mode
        self.img_path = img_path
        self.processor = self.get_processor()
        self.label_list, self.name_list = self.get_list(img_path) # return pathes and labels
        self.image_list = self.load_img()
        self.num_img = len(self.image_list)
        self.n_classes = n_classes
        print("Load " + str(self.num_img) + " images")

    def get_list(self, file_path):
        labels = []
        img_pathes = []
        for class_ in os.listdir(self.img_path):
            for image in os.listdir(f"{self.img_path}/{class_}"):
                labels.append(class_)
                img_pathes.append(f"{self.img_path}/{class_}/{image}")
        return labels, img_pathes

    
    def load_img(self):
        img_list = []
        for i, path in enumerate(self.name_list):
            img = PIL.Image.open(path)
            img = img.convert('L')
            img_list.append(img)
        return img_list
    
    def get_processor(self):
        proc = []
        re_size = 32
        proc.append(transforms.Resize((re_size, re_size)))
        proc.append(transforms.ToTensor())
            
        return transforms.Compose(proc)

    def __getitem__(self, index):
        #ltransform = [transforms.ToTensor()]
        #ltransform = transforms.Compose(ltransform)
        processer = self.get_processor()
        img = processer(self.image_list[index])
        if self.mode == "gan":
            return img
        label = self.label_list[index]
        label_ = np.zeros(self.n_classes + 1)
        label_[int(label)] = 1.0
        label_ = np.array(label_)
        return img, label_

    def __len__(self):
        return self.num_img



Обучение целевой модели и атакующей

Открытый набор данных представляет собой изображение цифр от 0 до 4
Закрытый набор данных представляет собой изображение цифр от 5 до 9

In [4]:
batch_size=64
classifier_train_loader = GrayFolder(private_data_path,"classifier" ,5)
classifier_train_loader = torch.utils.data.DataLoader(classifier_train_loader,
                                              batch_size=batch_size,
                                              shuffle=True,
                    
                                              pin_memory=True)
gan_train_loader = GrayFolder(public_data_path,"gan" , 5)
gan_train_loader = torch.utils.data.DataLoader(gan_train_loader,
                                              batch_size=batch_size,
                                              shuffle=True,
                            
                                              pin_memory=True)

Load 21416 images
Load 20584 images


Обучение генератора картинок

In [5]:
import os
import time
import utils
import torch
import torchvision
from utils import *
from torch.nn import BCELoss
from torch.autograd import grad
import torchvision.utils as tvls
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from discri import DGWGAN32 # модель дискриминатора
from generator import GeneratorMNIST # модель генератора

def freeze(net):
    for p in net.parameters():
        p.requires_grad_(False) 

def unfreeze(net):
    for p in net.parameters():
        p.requires_grad_(True)

def gradient_penalty(x, y):
    # interpolation
    shape = [x.size(0)] + [1] * (x.dim() - 1)
    alpha = torch.rand(shape)
    z = x + alpha * (y - x)
    z.requires_grad = True

    o = DG(z)
    g = grad(o, z, grad_outputs = torch.ones(o.size()), create_graph = True)[0].view(z.size(0), -1)
    gp = ((g.norm(p = 2, dim = 1) - 1) ** 2).mean()

    return gp

log_path = "./background/attack_logs"
save_img_dir = "./background/gan_samples"
save_model_dir= "./background/attack_models"
log_file = "./background/attack_logs/GAN.txt"
utils.Tee(log_file, 'w')

---------------------Training [%s]------------------------------
Epoch:0 	 Time:220.49
Epoch:1 	 Time:219.72
Epoch:2 	 Time:220.14
Epoch:3 	 Time:225.74
Epoch:4 	 Time:222.64
Epoch:5 	 Time:220.23
Epoch:6 	 Time:219.84
Epoch:7 	 Time:220.29
Epoch:8 	 Time:217.65
Epoch:9 	 Time:220.00
Epoch:10 	 Time:216.55
Epoch:11 	 Time:218.29
Epoch:12 	 Time:219.80
Epoch:13 	 Time:26494.66
Epoch:14 	 Time:223.79
Epoch:15 	 Time:225.67
Epoch:16 	 Time:222.55
Epoch:17 	 Time:222.30
Epoch:18 	 Time:222.65
Epoch:19 	 Time:218.57
Epoch:20 	 Time:225.86
Epoch:21 	 Time:219.94
Epoch:22 	 Time:219.06
Epoch:23 	 Time:218.22
Epoch:24 	 Time:53130.73
Epoch:0 	 Time:65.21
Epoch:1 	 Time:65.34
Epoch:2 	 Time:65.06
Epoch:3 	 Time:63.45
Epoch:4 	 Time:63.51
Epoch:5 	 Time:64.61
Epoch:6 	 Time:64.54
Epoch:7 	 Time:65.12
Epoch:8 	 Time:65.16
Epoch:9 	 Time:64.67
Epoch:10 	 Time:65.54
Epoch:11 	 Time:67.49
Epoch:12 	 Time:63.55
Epoch:13 	 Time:63.10
Epoch:14 	 Time:62.99
Epoch:15 	 Time:64.02
Epoch:16 	 Time:64.13
Ep

In [6]:
file_path = 'train_gan_file'
model_name = 'GAN'
epochs=100
lr=0.0002
z_dim=100
n_critic=5
weight_decay=1e-4
from tqdm import tqdm

In [7]:
print("---------------------Training [%s]------------------------------")
dataloader = gan_train_loader
G = GeneratorMNIST(z_dim)
DG = DGWGAN32()
G = torch.nn.DataParallel(G)
DG = torch.nn.DataParallel(DG)
dg_optimizer = torch.optim.Adam(DG.parameters(), lr=lr, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
step = 0
for epoch in range(epochs):
    start = time.time()
    for imgs in  tqdm(dataloader):
        step += 1
        bs = imgs.size(0)  
        freeze(G)
        unfreeze(DG)
        z = torch.randn(bs, z_dim)
        f_imgs = G(z)
        r_logit = DG(imgs)
        f_logit = DG(f_imgs)
        
        wd = r_logit.mean() - f_logit.mean()  # Wasserstein-1 Distance
        gp = gradient_penalty(imgs.data, f_imgs.data)
        dg_loss = - wd + gp * 10.0
            
        dg_optimizer.zero_grad()
        dg_loss.backward()
        dg_optimizer.step()
            # train G
        if step % n_critic == 0:
            freeze(DG)
            unfreeze(G)
            z = torch.randn(bs, z_dim)
            f_imgs = G(z)
            logit_dg = DG(f_imgs)
            # calculate g_loss
            g_loss = - logit_dg.mean()
                
            g_optimizer.zero_grad()
            g_loss.backward()
            g_optimizer.step()

    end = time.time()
    interval = end - start
    print("Epoch:%d \t Time:%.2f" % (epoch, interval))
    z = torch.randn(32, z_dim)
    fake_image = G(z)
    save_tensor_images(fake_image.detach(), os.path.join(save_img_dir, "result_image_{}.png".format(epoch)), nrow = 8)
        
    torch.save({'state_dict':G.state_dict()}, os.path.join(save_model_dir, "MNIST_G.tar"))
    torch.save({'state_dict':DG.state_dict()}, os.path.join(save_model_dir, "MNIST_D.tar"))

  2%|█▌                                                                                | 6/322 [00:05<04:25,  1.19it/s]


KeyboardInterrupt: 

Обучение целевой модели

In [12]:
model_name = 'target'
epochs=100
lr=0.0002
z_dim=100
n_critic=5
weight_decay=1e-4
from tqdm import tqdm

In [13]:
from discri import DGWGAN32
import time
import numpy as np
DG = DGWGAN32()
from classify import MCNN # содержит модель целевой модели
save_model_dir= "./background/target_models"
dataloader = classifier_train_loader
target_model = MCNN(num_classes=6)
target_model = torch.nn.DataParallel(target_model)
optimizer = torch.optim.Adam(DG.parameters(), lr=lr, betas=(0.5, 0.999))
loss_f = nn.CrossEntropyLoss()
step = 0
for epoch in range(epochs):
    start = time.time()
    for imgs, labels in  tqdm(dataloader):
        step += 1
        pred = target_model(imgs)[-1]
        loss = loss_f(pred,labels)    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    end = time.time()
    interval = end - start
    print("Epoch:%d \t Time:%.2f" % (epoch, interval))     
    torch.save({'state_dict':target_model.state_dict()}, os.path.join(save_model_dir, "MNIST_Target  .tar"))
print("Sucessfully target trained")

100%|████████████████████████████████████████████████████████████████████████████████| 335/335 [01:10<00:00,  4.72it/s]


Проведение gmi-атаки на модель

In [21]:
import torch, os, time, random, generator, discri, classify, utils
import numpy as np 
import torch.nn as nn
import torchvision.utils as tvls

device = "cpu"

num_classes = 6
log_path = "./background/attack_logs"


def inversion_fit(G:torch.nn.DataParallel, # генеративная модель
                  D:torch.nn.DataParallel, # модель дискриминатор
                  T:torch.nn.DataParallel, # целевая модель 
                  iden:torch.Tensor, # начальные метки
                  lr=1e-2,
                  momentum=0.9,
                  lamda=100,
                  iter_times=1500,
                  clip_range=1):
	iden = iden.view(-1).long()
	criterion = nn.CrossEntropyLoss() # нужна для Total_Loss
	bs = iden.shape[0]
	
	G.eval() 
	D.eval()
	T.eval()

	max_score = torch.zeros(bs)
	max_iden = torch.zeros(bs)
	z_hat = torch.zeros(bs, 100)
	
	for random_seed in tqdm(range(5)):
		tf = time.time()
		
		torch.manual_seed(random_seed) 
		torch.cuda.manual_seed(random_seed) 
		np.random.seed(random_seed) 
		random.seed(random_seed)

		z = torch.randn(bs, 100).float() # начальный шум, необходимый генератору для производства картинок
		z.requires_grad = True
		v = torch.zeros(bs, 100).float()
			
		for i in range(iter_times):
			fake = G(z) # генерируем изображение
			label = D(fake # получаем метку класса от дискриминатора, обученного на открытом датасете
			out = T(fake)[-1] # получаем метку класса от целевой модели, обученной на закрытом датасете
			
			if z.grad is not None:
				z.grad.data.zero_()

			Prior_Loss = - label.mean()
			Iden_Loss = criterion(out, iden)
			Total_Loss = Prior_Loss + lamda * Iden_Loss # вычисляем ошибку как лин. комбинацию от ошибки дискриминатора и ошибки целевой модели

			Total_Loss.backward() # Back Propagation
			
			v_prev = v.clone()
			gradient = z.grad.data
			v = momentum * v - lr * gradient
			z = z + ( - momentum * v_prev + (1 + momentum) * v) # Ищем оптимальное зачение начального шума, опираясь на гиперпараметры и пред. значение
			z = torch.clamp(z.detach(), -clip_range, clip_range).float()
			z.requires_grad = True
            
			Prior_Loss_val = Prior_Loss.item()
			Iden_Loss_val = Iden_Loss.item()
			if (i+1) % 300 == 0:
				fake_img = G(z.detach())
				eval_prob = T(fake_img)[-1]
				eval_iden = torch.argmax(eval_prob, dim=1).view(-1)
				acc = iden.eq(eval_iden.long()).sum().item() * 1.0 / bs		# метрики и промежуточные примеры
		fake = G(z)
		score = T(fake)[-1]
		eval_iden = torch.argmax(score, dim=1).view(-1)
        
	return G(z) # возвращаем результат работы генератора

In [22]:
from tqdm import tqdm
generator_path = "./background/attack_models/MNIST_G.tar"
discriminator_path = "./background/attack_models/MNIST_D.tar" 
target_path= "./background/target_models/MNIST_Target  .tar"
# загружаем предобученные модели
G = generator.GeneratorMNIST()
G = nn.DataParallel(G)
ckp_G = torch.load(generator_path)['state_dict']
utils.load_my_state_dict(G, ckp_G)

D = discri.DGWGAN32()
D = nn.DataParallel(D)
ckp_D = torch.load(discriminator_path)['state_dict']
utils.load_my_state_dict(D, ckp_D)

T = classify.MCNN(6)
T = nn.DataParallel(T)
ckp_T = torch.load(target_path)['state_dict']
utils.load_my_state_dict(T, ckp_T)

iden = torch.zeros(5,6)
iden.reshape((5,6))
for j in range(5):
    for i in range(6):
        iden[j][i] = i 

In [23]:
fake = inversion_fit(G, D, T, iden # сама атака

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [39:36<00:00, 475.22s/it]


In [24]:
from utils import save_tensor_images # сохраним комментарии
result_dir = "./results"
save_tensor_images(fake.detach(), os.path.join(result_dir, "result_another_iden_iter_1500.png"), nrow = 8)

Статья: The Secret Revealer: Generative Model-Inversion Attacks Against Deep Neural 
Networksan https://arxiv.org/pdf/1911.07135.pdf

Репозиторий: https://github.com/AI-secure/GMI-Attack/tree/master